<a href="https://colab.research.google.com/github/roman6s/SCM_Fallstudie/blob/main/Fallstudie_Gleitender_Mittelwert_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie BESS

In [ ]:
! git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

fatal: destination path 'WS24_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [ ]:
! pip install -q pyscipopt

In [ ]:
import pandas as pd
from pyscipopt import Model, quicksum

## Optimierungsmodell

In [ ]:
scip = Model()

## Daten laden

In [ ]:
folder = "WS24_Supply_Chain_Optimierung/Daten/Fallstudie"

In [ ]:
# Preisprogonose
preisprognose = pd.read_excel(f"{folder}/Preisprognosen.xlsx")


##Indexmenge

$T = \{1, 2, \ldots, 24\} \quad \text{(Menge der Stunden des Folgetages)}$

In [ ]:
T = range (1,25)

##Parameter

In [ ]:
cap       = 40.0            # Nominelle Kapazität in [MWh]

In [ ]:
DoD       = 0.80            # Depth of Discharge

In [ ]:
SOC_min   = cap*(1 - DoD)   # Minimaler Ladezustand in [MWh]

In [ ]:
SOC_max   = cap             # Maximaler Ladezustand in [MWh]

In [ ]:
c_rate    = 0.5             # C-Rate

In [ ]:
eta_rte = 0.975             # Round-trip Efficiency (einfache Richtung)

In [ ]:
eta_wr    = 0.985           # Wirkungsgrad Wechselrichter

In [ ]:
cyclecost = 1500.0          # Fixkosten pro Zyklusdurchlauf

In [ ]:
maxCycles = 2.0             # Maximal durchführbare Zyklen

In [ ]:
p_Markt_last_10 = preisprognose.groupby("Stunde")["Strompreis"].apply(lambda x: x.tail(10).mean()).tolist()


In [ ]:
p_Markt_last_10

[85.1,
 70.5,
 70.5,
 70.2,
 64.4,
 71.3,
 74.7,
 100.7,
 83.4,
 66.6,
 36.8,
 27.7,
 4.1,
 5.5,
 2.0,
 17.2,
 11.9,
 65.9,
 77.1,
 109.9,
 145.9,
 118.8,
 102.8,
 84.0]

In [ ]:
# Lade- und Entladeverluste trennen
# RTE für jede Richtung:
eta_charge    = eta_rte * eta_wr  # Lade-Verlust
eta_discharge = eta_rte * eta_wr  # Entlade-Verlust

In [ ]:
# Zählvariable für verbrauchte Zyklen
Z = scip.addVar(
      vtype="CONTINUOUS",
      lb=0,
      ub=maxCycles,
      name="cycles_used"
    )

##Entscheidungsvariablen

$\text{SOC}_t \in [8, 40] \quad [\text{MWh}], \text{ Ladezustand der Batterie am Ende von Stunde } t$


In [ ]:
# State of Charge je Stunde
SOC = {t: scip.addVar(
             vtype="CONTINUOUS",
             lb=SOC_min,
             ub=SOC_max,
             name=f"SOC_{t}"
          )
       for t in T}

$0 \leq \text{charge}_t \leq 20 \quad [\text{MWh}], \text{ Ladeleistung in Stunde } t.$


In [ ]:
# Ladestrom je Stunde (jeweils in MWh, nicht negativ)
charge = {t: scip.addVar(
               vtype="CONTINUOUS",
               lb=0,
               ub=c_rate * cap,
               name=f"charge_{t}"
            )
          for t in T}

$0 \leq \text{discharge}_t \leq 20 \quad [\text{MWh}], \text{ Entladeleistung in Stunde  } t.$


In [ ]:
# Entladestrom je Stunde (jeweils in MWh, nicht negativ)
discharge = {t: scip.addVar(
                  vtype="CONTINUOUS",
                  lb=0,
                  ub=c_rate * cap,
                  name=f"discharge_{t}"
               )
             for t in T}

##Nebenbedingungen

$\text{SOC}_1 = 0.5 \cdot \text{cap}, \quad \text{SOC}_{24} = 0.5 \cdot \text{cap}$


In [ ]:
# (1) Anfangs- und End-SOC = 50% Kapazität
scip.addCons(SOC[1] == 0.5 * cap)
scip.addCons(SOC[24] == 0.5 * cap)

# Verhindert Entladung bei t=24
scip.addCons(discharge[24] == 0)

c3

$\text{SOC}_t = \text{SOC}_{t-1} + \eta_{\text{charge}} \cdot \text{charge}_{t-1} - \frac{1}{\eta_{\text{discharge}}} \cdot \text{discharge}_{t-1}$


In [ ]:
# (2) Ladezustandsdynamik
for t in T:
    if t == 1:
        continue
    scip.addCons(
        SOC[t]
        == SOC[t-1]
           + eta_charge     * charge[t-1]
           - (1/eta_discharge)* discharge[t-1]
    )

$
\frac{\sum_{t \in T} (\text{charge}_t + \text{discharge}_t)}{2 \cdot \text{cap} \cdot \text{DoD}} \leq Z \leq \text{maxCycles}
$





In [ ]:
# (3) Zyklenbegrenzung
#     Ein Vollzyklus = 2*(cap * DoD). Also berechnen wir den gesamten "Throughput"
#     und setzen Z >= Throughput / [2 * (cap*DoD)].
throughput = quicksum(charge[t] + discharge[t] for t in T)
scip.addCons(
    Z >= throughput / (2.0 * cap * DoD)
)
# Z soll <= 2 sein
scip.addCons(
    Z <= maxCycles
)

c28

##Zielfunktion

$Max \quad \underbrace{\sum_{t \in T} \left[ p_t \cdot (\text{discharge}_t) - p_t \cdot (\text{charge}_t) \right]}_{\text{Erlöse - Kosten für Strom}}
-
\underbrace{\text{cost} \cdot Z}_{\text{Zykluskosten}}
$


In [ ]:
#   Erlöse - Kosten pro Zyklus
scip.setObjective(quicksum(
    p_Markt_last_10[t-1] * discharge[t] - p_Markt_last_10[t-1] * charge[t]
    for t in T
) - cyclecost * Z, sense="maximize")

##Berechnung des Ergebnis

In [ ]:
# Modell lösen
scip.optimize()

In [ ]:
# Ausgabe der Ergebnisse
print(f"Optimaler Zielfunktionswert: {scip.getObjVal():.2f} €")

print("\nStundenweise Lösung:")
for t in T:
    soc_val     = scip.getVal(SOC[t])
    chg_val     = scip.getVal(charge[t])
    dis_val     = scip.getVal(discharge[t])
    print(f"  t={t:2d}: "
          f"SOC={soc_val:5.2f}  "
          f"charge={chg_val:5.2f}  "
          f"discharge={dis_val:5.2f}  "
          f"Preis={p_Markt_last_10[t-1]:6.2f}")

print(f"\nEffektiv genutzte Zyklen: {scip.getVal(Z):.2f}")

Optimaler Zielfunktionswert: 2367.05 €

Stundenweise Lösung:
  t= 1: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 85.10
  t= 2: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 70.50
  t= 3: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 70.50
  t= 4: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 70.20
  t= 5: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 64.40
  t= 6: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 71.30
  t= 7: SOC=20.00  charge= 0.00  discharge= 0.00  Preis= 74.70
  t= 8: SOC=20.00  charge= 0.00  discharge=11.52  Preis=100.70
  t= 9: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 83.40
  t=10: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 66.60
  t=11: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 36.80
  t=12: SOC= 8.00  charge= 0.00  discharge= 0.00  Preis= 27.70
  t=13: SOC= 8.00  charge=13.32  discharge= 0.00  Preis=  4.10
  t=14: SOC=20.79  charge= 0.00  discharge= 0.00  Preis=  5.50
  t=15: SOC=20.79  charge=20.00  discharge= 0.00  Preis= 